In [1]:
import yaml
import dotenv
import os
import pandas as pd

In [2]:
dotenv.load_dotenv()
base_path = os.getenv("BASE_PATH")
config_path = os.path.join(base_path, "config.yml")
save_dir_dataset = os.path.join(base_path, "audio_set")
base_path

'D:\\audio_cls_coursework'

In [3]:
with open(config_path, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

dataset_name = list(config["datasets"].keys())[0]
print(dataset_name)
url_train = config["datasets"][dataset_name]["train"]
print(url_train)
labels_item = config["labels"].items()
print(labels_item)

audio_set
http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/balanced_train_segments.csv
dict_items([('siren', '/m/07rwj'), ('air_raid_siren', '/m/01hsr_'), ('alarm', '/m/0c3f7m'), ('explosion', '/m/0k4j'), ('gunshot', '/m/03_1v'), ('emergency_vehicle', '/m/04zjc')])


In [4]:
from scripts.downloading_datasets import download_dataset

await download_dataset(
    save_dir_dataset,
    url_dataset=url_train,
    dataset_name=dataset_name,
)
save_path = os.path.join(base_path, dataset_name, f"{dataset_name}.csv")

In [5]:
df = pd.read_csv(
    save_path,
    comment="#",
    header=None,
    names=["YTID", "start_seconds", "end_seconds", "positive_labels"],
    sep=", ",
    engine="python"
)

df

,YTID,start_seconds,end_seconds,positive_labels
0,--PJHxphWEs,30.0,40.0,"""/m/09x0r,/t/dd00088"""
1,--ZhevVpy1s,50.0,60.0,"""/m/012xff"""
2,--aE2O5G5WE,0.0,10.0,"""/m/03fwl,/m/04rlf,/m/09x0r"""
3,--aO5cdqSAg,30.0,40.0,"""/t/dd00003,/t/dd00005"""
4,--aaILOrkII,200.0,210.0,"""/m/032s66,/m/073cg4"""
...,...,...,...,...
22155,zyqg4pYEioQ,20.0,30.0,"""/m/09x0r,/m/0llzx"""
22156,zz0ddNfz0h0,30.0,40.0,"""/m/012f08,/m/03cl9h,/m/07yv9,/m/0k4j"""
22157,zz8TGV83nkE,80.0,90.0,"""/m/012f08,/m/02mk9,/m/04_sv,/m/07yv9"""
22158,zzlK8KDqlr0,370.0,380.0,"""/m/01m2v,/m/07qc9xj,/m/09x0r,/t/dd00125"""


In [6]:
from scripts.fltering_dataset import fiter_dataset

df = fiter_dataset(df, labels_item)


100%|██████████| 6/6 [00:00<00:00, 160.07it/s]


,YTID,start_seconds,end_seconds,positive_labels
127,-FVBrg2qeIA,90.0,100.0,"""/m/0342h,/m/04rlf,/m/04szw,/m/07rwj3x,/m/09x0..."
166,-K4R9SXZZsk,10.0,20.0,"""/m/012f08,/m/07yv9,/m/096m7z,/m/0k4j"""
336,-jBWkHhQNew,30.0,40.0,"""/m/02mk9,/m/07pb8fc,/m/07q2z82,/m/07yv9,/m/0k..."
352,-lYg4hm25hU,30.0,40.0,"""/m/04rlf,/m/07q4ntr,/m/07rwj3x,/m/09x0r"""
360,-mFneOHpwak,270.0,280.0,"""/m/04rlf,/m/07rwj3x"""
...,...,...,...,...
21939,zNYHzcIRMFY,30.0,40.0,"""/m/07pp_mv,/m/0c3f7m"""
21967,zRfi-kb9S2s,10.0,20.0,"""/m/07rknqz,/m/07yv9,/m/0h9mv,/m/0k4j,/m/0ltv"""
21996,zWQJrAaHsFg,260.0,270.0,"""/m/015lz1,/m/04rlf,/m/07rwj3x"""
22062,zhH3QeoBZDI,320.0,330.0,"""/m/01hsr_"""


In [7]:
df.shape

(537, 4)

In [ ]:
from tqdm import tqdm
import yt_dlp


def download_segment(ytid, start_sec, end_sec):
    output_path = os.path.join(base_path, dataset_name, 'clip',
                               f"{ytid}_{int(start_sec)}_{int(end_sec)}.wav")
    url = f"https://www.youtube.com/watch?v={ytid}"

    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": output_path,
        "quiet": True,
        "postprocessors": [{
            "key": "FFmpegExtractAudio",
            "preferredcodec": "wav",
            "preferredquality": "192",
        }],
        "postprocessor_args": [
            "-ss", str(start_sec),
            "-to", str(end_sec)
        ]
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        return f"{ytid}: OK"
    except Exception as e:
        return f"{ytid}: Помилка {e}"


for _, row in tqdm(df.iterrows(), total=len(df)):
    ytid = row["YTID"].replace("--", "")
    start_sec = row["start_seconds"]
    end_sec = row["end_seconds"]
    print(download_segment(ytid, start_sec, end_sec))

  0%|          | 1/537 [00:02<25:17,  2.83s/it]

-FVBrg2qeIA: Помилка ERROR: [youtube] -FVBrg2qeIA: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


         player = https://www.youtube.com/s/player/87644c66/player_ias.vflset/en_US/base.js
         n = Zz62LlXkQ9HfuI88AgK ; player = https://www.youtube.com/s/player/87644c66/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


  0%|          | 2/537 [00:07<34:14,  3.84s/it]

-K4R9SXZZsk: OK


...

APit6hIABQA: Помилка ERROR: [youtube] APit6hIABQA: Requested format is not available. Use --list-formats for a list of available formats


         player = https://www.youtube.com/s/player/c6d7bdc9/player_ias.vflset/en_US/base.js
         n = FvCq4q25Uu2MQk-C ; player = https://www.youtube.com/s/player/c6d7bdc9/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U
[download] Got error: HTTP Error 403: Forbidden
[download] Got error: HTTP Error 403: Forbidden
ERROR: [youtube] Aax8cjFZZ2o: Requested format is not available. Use --list-formats for a list of available formats
 20%|██        | 110/537 [17:26<30:39,  4.31s/it]

Aax8cjFZZ2o: Помилка ERROR: [youtube] Aax8cjFZZ2o: Requested format is not available. Use --list-formats for a list of available formats


         player = https://www.youtube.com/s/player/87644c66/player_ias.vflset/en_US/base.js
         n = 2pLWMIok8-doMQ6l5eW ; player = https://www.youtube.com/s/player/87644c66/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U
[download] Got error: HTTP Error 403: Forbidden
[download] Got error: HTTP Error 403: Forbidden
ERROR: [youtube] Ab51oVS98Qo: Requested format is not available. Use --list-formats for a list of available formats
 21%|██        | 111/537 [17:31<32:05,  4.52s/it]

Ab51oVS98Qo: Помилка ERROR: [youtube] Ab51oVS98Qo: Requested format is not available. Use --list-formats for a list of available formats
